In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
cf1_results = pd.read_csv('/content/cf1_results.csv')
cf1_results_ltn = pd.read_csv('/content/cf1_results_ltn.csv')

# Calculate the last epoch of the normal model
last_normal_epoch = cf1_results['Epoch'].max()

# Adjust the LTN model epochs to start right after the last epoch of the normal model
cf1_results_ltn['Adjusted Epoch'] = cf1_results_ltn['Epoch'] + last_normal_epoch

# Group by Epoch and calculate mean accuracy for the normal model
avg_cf1_results = cf1_results.groupby('Epoch').agg({'Validation Accuracy': 'mean'}).reset_index()

# Find the maximum test accuracy for each fold and speed in the LTN model
max_accuracy_per_fold_speed = cf1_results_ltn.groupby(['Fold', 'Speed', 'Adjusted Epoch']).agg({'test_accuracy': 'max'}).reset_index()

# Calculate the mean of these maximum accuracies for each adjusted epoch
avg_max_accuracy_per_epoch = max_accuracy_per_fold_speed.groupby('Adjusted Epoch').agg({'test_accuracy': 'mean'}).reset_index()

# Combine averaged data for a continuous plot
combined_avg_results = pd.concat([
    avg_cf1_results.rename(columns={'Validation Accuracy': 'Accuracy', 'Epoch': 'Extended Epoch'}),
    avg_max_accuracy_per_epoch.rename(columns={'test_accuracy': 'Accuracy', 'Adjusted Epoch': 'Extended Epoch'})
])

# Plotting
plt.figure(figsize=(14, 8))
plt.plot(combined_avg_results['Extended Epoch'], combined_avg_results['Accuracy'], label='Average Model Accuracy', marker='o')
plt.title('Average Model Accuracy Across Extended Epochs')
plt.xlabel('Extended Epoch (Normal + LTN)')
plt.ylabel('Average Test Accuracy')
plt.axvline(x=last_normal_epoch, color='r', linestyle='--', label='Start of LTN Model')
plt.legend()
plt.grid(True)
plt.show()
